In [2]:
import json
data_path = "ALQAC_2025_data/alqac25_law.json"
with open(data_path, "r", encoding="utf-8") as f:
  data = json.load(f)

In [3]:
# Count the number of articles and rename their IDs
articles = []
for doc in data:
    for article in doc["articles"]:
        # Rename article ID
        new_article = article.copy()
        new_article["id"] = f'{doc["id"]} - {article["id"]}'
        articles.append(new_article)

print(len(articles))

3368


In [13]:
articles[50:52]

[{'id': 'Bộ luật dân sự - 433',
  'text': 'Giá và phương thức thanh toán\n\n1. Giá, phương thức thanh toán do các bên thoả thuận hoặc do người thứ ba xác định theo yêu cầu của các bên. Trường hợp pháp luật quy định giá, phương thức thanh toán phải theo quy định của cơ quan nhà nước có thẩm quyền thì thỏa thuận của các bên phải phù hợp với quy định đó.\n\n2. Trường hợp không có thỏa thuận hoặc thỏa thuận không rõ ràng về giá, phương thức thanh toán thì giá được xác định theo giá thị trường, phương thức thanh toán được xác định theo tập quán tại địa điểm và thời điểm giao kết hợp đồng.'},
 {'text': 'Giá dịch vụ viễn thông\n1. Giá dịch vụ viễn thông gồm giá dịch vụ viễn thông áp dụng đối với người sử dụng dịch vụ viễn thông và giá dịch vụ giữa các doanh nghiệp viễn thông.\n2. Giá dịch vụ viễn thông áp dụng đối với người sử dụng dịch vụ viễn thông là giá mà người sử dụng dịch vụ viễn thông thanh toán cho doanh nghiệp viễn thông khi sử dụng dịch vụ viễn thông do doanh nghiệp viễn thông cung

In [ ]:
import random
from tqdm import tqdm
import re
import os
from itertools import islice
import re
from openai import OpenAI

output_json_path = "ALQAC_2025_data/alqac_2025_qg.json"


# model_name = "DeepSeek-R1-Distill-Llama-8B"
model_name = "Qwen2.5-7B-Instruct"



class FPTLlm:
    def __init__(self, model_name: str = "Qwen2.5-7B-Instruct"):
        self.client = OpenAI(api_key="sk-gZOpFw0WOZhUF4fXBXoLsQ",
                             base_url="https://mkp-api.fptcloud.com")
        self.model_name = model_name

    def complete(self, input: str) -> str:
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[
                {
                    "role": "user",
                    "content": input
                }
            ],
            max_tokens=1000,
            temperature=0.7,
            # chat_template_kwargs={"enable_thinking": false}
        )
        return response.choices[0].message.content.strip()




def chunk_list(lst, chunk_size):
    """Yield successive chunk_size-sized chunks from a list."""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

def extract_json_from_markdown(text):
    """
    Finds and parses a JSON object embedded in a markdown code block.
    Returns a dictionary or None.
    """
    match = re.search(r'```json\s*\n(.*?)\n\s*```', text, re.DOTALL)
    if not match:
        match = re.search(r'```json\s*(.*?)\s*```', text, re.DOTALL)

    if match:
        json_string = match.group(1).strip()
        try:
            return json.loads(json_string)
        except json.JSONDecodeError:
            print(f"Error: Failed to decode JSON.\nContent: {json_string}")
            return None
    else:
        # Fallback for plain JSON without markdown, as in previous versions
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            # print("Error: No JSON markdown block or valid JSON found.")
            return None

def generate_queries(
    corpus,
    num_questions_per_chunk=3,
    docs_per_chunk=3,
    num_of_pairs=None,
    prompt_template=None,
    verbose=False,
):
    """
    Generate questions from random document chunks in a list-based corpus.
    """
    llm = FPTLlm(model_name=model_name)
    output = []

    random.shuffle(corpus)
    all_chunks = list(chunk_list(corpus, docs_per_chunk))

    # Each "pair" means: a chunk of docs_per_chunk documents (not pairs of questions)
    if num_of_pairs is not None and num_of_pairs < len(all_chunks):
        all_chunks = random.sample(all_chunks, num_of_pairs)

    last_response = None

    if os.path.exists(output_json_path):
        with open(output_json_path, "r", encoding="utf-8") as f:
            try:
                existing_data = json.load(f)
            except Exception:
                existing_data = []
    else:
        existing_data = []

    for chunk in tqdm(all_chunks, desc="Generating queries"):
        node_ids = [doc["id"] for doc in chunk]
        context_str = "\n\n".join(doc["text"] for doc in chunk)

        query = prompt_template.format(
            context_str=context_str,
            num_questions_per_chunk=num_questions_per_chunk
        )
        response = llm.complete(query)
        last_response = response

        extracted_data = extract_json_from_markdown(str(response))

        questions = []
        if extracted_data and "question" in extracted_data:
            question_value = extracted_data["question"]
            if isinstance(question_value, list):
                questions.extend(question_value)
            elif isinstance(question_value, str):
                questions.append(question_value)

        for question in questions:
            if not question:
                continue
            example = {"anchor": question, "positive": context_str, "relevant_docs": node_ids}
            output.append(example)
            existing_data.append(example)
            if verbose:
                print(f"Generated: {question} -> {node_ids}")
            # Write each new example appended to JSON file
            with open(output_json_path, "w", encoding="utf-8") as f:
                json.dump(existing_data, f, ensure_ascii=False, indent=2)

    return last_response, output

In [138]:
prompt = """
Bạn là một hệ thống tạo đề kiểm tra.

Dưới đây là một đoạn văn bản được cung cấp. Nhiệm vụ của bạn là **tạo ra các câu hỏi** có liên quan đến nội dung văn bản. 
**Không sử dụng kiến thức bên ngoài**.

---------------------
{context_str}
---------------------

## Yêu cầu:

1. Đọc kỹ và xác định các thông tin quan trọng trong đoạn văn bản.
2. Dựa trên những thông tin này, hãy tạo ra các câu hỏi ngắn gọn, rõ ràng, có thể dùng để kiểm tra kiến thức.
4. Ghi chú: Không cần phân tích nội dung, không viết lời giải thích, không thêm nhận xét cá nhân.

## Định dạng đầu ra:

Chỉ xuất ra **một đoạn JSON duy nhất** có cấu trúc như sau:

```json
{{"question": "Câu hỏi thứ nhất?"}}
```
Ví dụ 1:
---------------------
Nguyên tắc trong hợp tác quốc tế về phòng, chống ma túy
1. Nhà nước Việt Nam thực hiện điều ước quốc tế về phòng, chống ma túy và điều ước quốc tế khác có liên quan mà nước Cộng hòa xã hội chủ nghĩa Việt Nam
là thành viên trên nguyên tắc tôn trọng độc lập, chủ quyền, toàn vẹn lãnh thổ và các bên cùng có lợi; hợp tác với các nước, tổ chức quốc tế, tổ chức,
cá nhân nước ngoài trong hoạt động phòng, chống ma túy.\n\n2. Trên cơ sở điều ước quốc tế mà nước Cộng hòa xã hội chủ nghĩa Việt Nam là thành viên,
quy định của Luật này và quy định khác của pháp luật có liên quan, thỏa thuận quốc tế đã ký kết, cơ quan có thẩm quyền của Việt Nam
thực hiện chương trình hợp tác về phòng, chống ma túy với cơ quan có liên quan của các nước, tổ chức quốc tế, tổ chức, cá nhân nước ngoài
---------------------
Đầu ra mong đợi:
```json
{{"question": "Nhà nước Việt Nam hợp tác quốc tế về phòng, chống ma túy trên nguyên tắc nào?"}}
```

Ví dụ 2:
---------------------
Nguyên tắc trong hợp tác quốc tế về phòng, chống ma túy
1. Nhà nước Việt Nam thực hiện điều ước quốc tế về phòng, chống ma túy và điều ước quốc tế khác có liên quan mà nước Cộng hòa xã hội chủ nghĩa Việt Nam
là thành viên trên nguyên tắc tôn trọng độc lập, chủ quyền, toàn vẹn lãnh thổ và các bên cùng có lợi; hợp tác với các nước, tổ chức quốc tế, tổ chức,
cá nhân nước ngoài trong hoạt động phòng, chống ma túy.\n\n2. Trên cơ sở điều ước quốc tế mà nước Cộng hòa xã hội chủ nghĩa Việt Nam là thành viên,
quy định của Luật này và quy định khác của pháp luật có liên quan, thỏa thuận quốc tế đã ký kết, cơ quan có thẩm quyền của Việt Nam
thực hiện chương trình hợp tác về phòng, chống ma túy với cơ quan có liên quan của các nước, tổ chức quốc tế, tổ chức, cá nhân nước ngoài
---------------------

Đầu ra mong đợi:
```json
{{"question": "Cơ quan có thẩm quyền của Việt Nam thực hiện chương trình hợp tác phòng, chống ma túy với các đối tượng nào?"}}
```
"""

In [136]:
# Generate question relevant to 1 articles only
queries, generated_pairs = generate_queries(
    corpus=articles,
    num_questions_per_chunk=1,
    docs_per_chunk=1,
    num_of_pairs=4000,  # Number of question-doc groups to generate
    prompt_template=prompt,
    verbose=True,
)

Generating queries:   0%|          | 1/3368 [00:00<55:22,  1.01it/s]

Generated: Quyết định phê duyệt kết quả thẩm định báo cáo đánh giá tác động môi trường là căn cứ để cơ quan có thẩm quyền thực hiện việc cấp giấy phép khai thác khoáng sản đối với dự án đầu tư khai thác khoáng sản? -> ['Luật Bảo vệ môi trường - 36']


Generating queries:   0%|          | 2/3368 [00:01<36:11,  1.55it/s]

Generated: Các hành vi nào bị nghiêm cấm trong quá trình cung cấp thông tin? -> ['Luật Tiếp cận thông tin - 11']


Generating queries:   0%|          | 3/3368 [00:01<32:51,  1.71it/s]

...

Generated: Khi ai gây thiệt hại cố ý trong khi thực hiện công việc không có ủy quyền thì phải bồi thường? -> ['Bộ luật dân sự - 577']


Generating queries: 100%|█████████▉| 3367/3368 [27:59<00:00,  1.81it/s]

Generated: Cơ quan, tổ chức, cá nhân có trách nhiệm hỗ trợ như thế nào trong hoạt động giáo dục và nghiên cứu khoa học? -> ['Luật Giáo dục - 93']


Generating queries: 100%|██████████| 3368/3368 [27:59<00:00,  2.00it/s]

Generated: Cha và mẹ có nghĩa vụ và quyền gì đối với con chưa thành niên? -> ['Luật Hôn nhân và gia đình - 71']


In [137]:
# Gerenate question relevant to 2 articles

queries, generated_pairs = generate_queries(
    corpus=articles,
    num_questions_per_chunk=1,
    docs_per_chunk=2,
    num_of_pairs=200,  # Number of question-doc groups to generate
    prompt_template=prompt,
    verbose=True,
)

Generating queries:   0%|          | 1/200 [00:00<02:17,  1.45it/s]

Generated: Trường hợp nào dưới đây không phải là lý do để hoãn phiên tòa? -> ['Luật Tố tụng hành chính - 162', 'Luật Bảo vệ môi trường - 71']


Generating queries:   1%|          | 2/200 [00:01<01:51,  1.78it/s]

Generated: Dự án đầu tư xây dựng nhà lưu trú công nhân trong khu công nghiệp phải đáp ứng yêu cầu nào? -> ['Luật Nhà ở - 95', 'Bộ luật dân sự - 41']


Generating queries:   2%|▏         | 3/200 [00:01<01:45,  1.87it/s]

...

Generated: Trong thời hạn chuẩn bị xét xử sơ thẩm, việc đối thoại giữa tòa án và đương sự phải tuân theo nguyên tắc nào? -> ['Luật Tố tụng hành chính - 134', 'Luật Khám bệnh, chữa bệnh - 7']


Generating queries: 100%|█████████▉| 199/200 [01:46<00:00,  1.69it/s]

Generated: Kế hoạch quản lý chất lượng môi trường nước mặt đối với sông, hồ liên tỉnh phải phù hợp với gì? -> ['Luật Bảo vệ môi trường - 9', 'Bộ luật dân sự - 51']


Generating queries: 100%|██████████| 200/200 [01:46<00:00,  1.87it/s]

Generated: Tổ chức, cá nhân xuất khẩu phim cần có giấy phép gì? -> ['Luật Điện ảnh - 17', 'Luật An ninh mạng - 14']


In [139]:
# Gerenate question relevant to 3 articles

queries, generated_pairs = generate_queries(
    corpus=articles,
    num_questions_per_chunk=1,
    docs_per_chunk=3,
    num_of_pairs=50,  # Number of question-doc groups to generate
    prompt_template=prompt,
    verbose=True,
)

Generating queries:   2%|▏         | 1/50 [00:00<00:42,  1.17it/s]

Generated: Hành lang bảo vệ nguồn nước được lập để đạt mục đích gì? -> ['Luật Tài nguyên nước - 23', 'Luật Bảo vệ quyền lợi người tiêu dùng - 26', 'Luật Giá - 67']


Generating queries:   4%|▍         | 2/50 [00:01<00:34,  1.40it/s]

Generated: Trong thời hạn bao lâu kể từ ngày thụ lý vụ án, Thẩm phán phải ra quyết định đưa vụ án ra xét xử hoặc đình chỉ vụ án và trả lại đơn khởi kiện? -> ['Luật Tố tụng hành chính - 199', 'Luật Khám bệnh, chữa bệnh - 61', 'Luật Tố tụng hành chính - 175']


Generating queries:   6%|▌         | 3/50 [00:02<00:36,  1.30it/s]

...

Generated: Quy chuẩn kỹ thuật môi trường về chất lượng môi trường xung quanh quy định mức giá trị giới hạn cho phép của các thông số môi trường dựa trên mục đích sử dụng của thành phần môi trường như thế nào? -> ['Luật Bảo vệ môi trường - 100', 'Luật Tố tụng hành chính - 251', 'Bộ luật dân sự - 582']


Generating queries:  98%|█████████▊| 49/50 [00:26<00:00,  1.73it/s]

Generated: Cơ quan nhà nước có thẩm quyền quyết định dừng phổ biến phim là ai? -> ['Luật Điện ảnh - 30', 'Luật Bảo vệ môi trường - 46', 'Luật Tố tụng hành chính - 263']


Generating queries: 100%|██████████| 50/50 [00:27<00:00,  1.85it/s]

Generated: Bộ luật dân sự điều chỉnh các quan hệ dân sự như thế nào? -> ['Bộ luật dân sự - 4', 'Bộ luật dân sự - 43', 'Luật Bảo vệ quyền lợi người tiêu dùng - 32']


In [140]:
all_chunks = list(chunk_list(articles, 2))

In [ ]:
all_chunks[1]

[{'id': 'Bộ luật dân sự - 71',
  'text': 'Tuyên bố chết\n\n1. Người có quyền, lợi ích liên quan có thể yêu cầu Tòa án ra quyết định tuyên bố một người là đã chết trong trường hợp sau đây:\n\na) Sau 03 năm, kể từ ngày quyết định tuyên bố mất tích của Tòa án có hiệu lực pháp luật mà vẫn không có tin tức xác thực là còn sống;\n\nb) Biệt tích trong chiến tranh sau 05 năm, kể từ ngày chiến tranh kết thúc mà vẫn không có tin tức xác thực là còn sống;\n\nc) Bị tai nạn hoặc thảm họa, thiên tai mà sau 02 năm, kể từ ngày tai nạn hoặc thảm hoạ, thiên tai đó chấm dứt vẫn không có tin tức xác thực là còn sống, trừ trường hợp pháp luật có quy định khác;\n\nd) Biệt tích 05 năm liền trở lên và không có tin tức xác thực là còn sống; thời hạn này được tính theo quy định tại khoản 1 Điều 68 của Bộ luật này.\n\n2. Căn cứ vào các trường hợp quy định tại khoản 1 Điều này, Tòa án xác định ngày chết của người bị tuyên bố là đã chết.\n\n3. Quyết định của Tòa án tuyên bố một người là đã chết phải được gửi cho Ủ

: 